### MID 9
### Taking ant1.3 as training data.
### ant1.4 as testing data.
### Min max scaling done to few columns: ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
### Oversampling and smote methods used to increase size of training data.
### RNN model used, model type is 4 as per the BTP documentation spreadsheet.
### np.rint() used on predictions

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
import tensorflow as tf
import csv

In [2]:
# Loading data. 
train_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv"
test_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"
train_data_name = "ant-1.3"
test_data_name = "ant-1.4"
ant_1_3 = pd.read_csv(train_data_path)
ant_1_4 = pd.read_csv(test_data_path)
files = ["/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv", "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"]
combined_data = pd.concat(map(pd.read_csv, files))

In [3]:
# Applying Min Max Scaling.
scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
combined_data[cols_to_norm] = MinMaxScaler().fit_transform(combined_data[cols_to_norm])

ant_1_3_scaled = combined_data[combined_data['version'] == 1.3] 
ant_1_4_scaled = combined_data[combined_data['version'] == 1.4] 

In [4]:
# Getting training and test data.
X_train = ant_1_3_scaled[cols_to_norm]
X_train = np.array(X_train)
Y_train = ant_1_3['bug']

X_test = ant_1_4_scaled[cols_to_norm]
X_test = np.array(X_test)
Y_test = ant_1_4['bug']

In [5]:
# Applying over sampling and SMOTE to training data for augmentation.
ros = RandomOverSampler(random_state=0)
X_train, Y_train = ros.fit_resample(X_train, Y_train)

smt = SMOTE()
X_train, Y_train = smt.fit_resample(X_train, Y_train)

train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [6]:
train_x 

array([[[0.14285714, 0.6       , 0.05      , ..., 0.16595094,
         0.08571429, 0.20908494]],

       [[0.18181818, 0.        , 0.025     , ..., 0.08097907,
         0.17142857, 0.26990307]],

       [[0.03896104, 0.2       , 0.        , ..., 0.08326661,
         0.02857143, 0.1095285 ]],

       ...,

       [[0.22077922, 0.        , 0.        , ..., 0.13535534,
         0.05714286, 0.16428454]],

       [[0.22077922, 0.        , 0.        , ..., 0.13535534,
         0.05714286, 0.16428454]],

       [[0.07792208, 0.4       , 0.        , ..., 0.15132106,
         0.05714286, 0.16428454]]])

In [7]:
Y_train

0      0
1      2
2      0
3      0
4      0
      ..
415    3
416    3
417    3
418    3
419    3
Name: bug, Length: 420, dtype: int64

In [8]:
train_y = Y_train.to_numpy()
train_y

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 3, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [9]:
test_x

array([[[0.18181818, 0.4       , 0.        , ..., 0.12421366,
         0.11428571, 0.25815673]],

       [[0.06493506, 0.4       , 0.        , ..., 0.25748599,
         0.02857143, 0.13142763]],

       [[0.16883117, 0.        , 0.025     , ..., 0.03473548,
         0.02857143, 0.12636767]],

       ...,

       [[0.2987013 , 0.        , 0.        , ..., 0.2197236 ,
         0.68571429, 0.37856087]],

       [[0.14285714, 0.        , 0.2       , ..., 0.37338962,
         0.62857143, 0.76168885]],

       [[0.05194805, 0.2       , 0.        , ..., 0.06485188,
         0.14285714, 0.28749795]]])

In [10]:
Y_test

0      0
1      0
2      0
3      0
4      0
      ..
173    0
174    0
175    0
176    1
177    0
Name: bug, Length: 178, dtype: int64

In [11]:
test_y = Y_test.to_numpy()
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0])

In [12]:
# Designing and initializing the model.
model = Sequential()
model.add(SimpleRNN(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(80, dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1, 100)            12100     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 1, 80)             14480     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 60)                8460      
_________________________________________________________________
dense (Dense)                (None, 1)                 61        
Total params: 35,101
Trainable params: 35,101
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100, batch_size = 128)

Epoch 1/100
4/4 [==============================] - 0s 5ms/step - loss: 2.8624 - mse: 2.8624 - mae: 1.3445 - root_mean_squared_error: 1.6919 - mean_squared_logarithmic_error: 0.6500
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4116 - mse: 1.4116 - mae: 0.9988 - root_mean_squared_error: 1.1881 - mean_squared_logarithmic_error: 0.2891
Epoch 3/100
4/4 [==============================] - 0s 5ms/step - loss: 1.4350 - mse: 1.4350 - mae: 0.9761 - root_mean_squared_error: 1.1979 - mean_squared_logarithmic_error: 0.3232
Epoch 4/100
4/4 [==============================] - 0s 5ms/step - loss: 1.3084 - mse: 1.3084 - mae: 0.9610 - root_mean_squared_error: 1.1439 - mean_squared_logarithmic_error: 0.2991
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 1.1684 - mse: 1.1684 - mae: 0.9331 - root_mean_squared_error: 1.0809 - mean_squared_logarithmic_error: 0.2469
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2690 - mse: 1.2690 - m

4/4 [==============================] - 0s 5ms/step - loss: 0.9384 - mse: 0.9384 - mae: 0.8376 - root_mean_squared_error: 0.9687 - mean_squared_logarithmic_error: 0.2020
Epoch 47/100
4/4 [==============================] - 0s 5ms/step - loss: 0.9659 - mse: 0.9659 - mae: 0.8451 - root_mean_squared_error: 0.9828 - mean_squared_logarithmic_error: 0.2141
Epoch 48/100
4/4 [==============================] - 0s 5ms/step - loss: 0.9320 - mse: 0.9320 - mae: 0.8174 - root_mean_squared_error: 0.9654 - mean_squared_logarithmic_error: 0.2181
Epoch 49/100
4/4 [==============================] - 0s 7ms/step - loss: 0.9688 - mse: 0.9688 - mae: 0.8433 - root_mean_squared_error: 0.9843 - mean_squared_logarithmic_error: 0.2306
Epoch 50/100
4/4 [==============================] - 0s 6ms/step - loss: 0.9272 - mse: 0.9272 - mae: 0.8178 - root_mean_squared_error: 0.9629 - mean_squared_logarithmic_error: 0.2089
Epoch 51/100
4/4 [==============================] - 0s 7ms/step - loss: 0.9723 - mse: 0.9723 - mae: 0.8

In [16]:
# Getting the predictions.
predictions_y = model.predict(test_x)
predictions_y

array([[1.6072546 ],
       [2.1477766 ],
       [0.77377445],
       [1.9239627 ],
       [1.6416126 ],
       [1.7778059 ],
       [1.4475752 ],
       [0.        ],
       [0.524423  ],
       [1.7588464 ],
       [1.5282265 ],
       [1.9072379 ],
       [0.80756897],
       [1.47969   ],
       [2.018737  ],
       [0.        ],
       [1.7273225 ],
       [0.16834114],
       [0.        ],
       [1.0171773 ],
       [0.9072643 ],
       [0.27821466],
       [0.8933983 ],
       [1.7508308 ],
       [1.9665291 ],
       [1.6367217 ],
       [1.0434351 ],
       [1.0920624 ],
       [1.7114413 ],
       [0.        ],
       [0.        ],
       [0.34577543],
       [0.06508484],
       [1.3132544 ],
       [0.290638  ],
       [0.        ],
       [1.4379394 ],
       [0.        ],
       [0.90239745],
       [1.4254798 ],
       [1.6591657 ],
       [0.        ],
       [0.        ],
       [2.005864  ],
       [0.        ],
       [0.04958551],
       [1.4366235 ],
       [1.948

In [17]:
# Rounding off the predictions to nearest
#integer as count of bugs is an integer.
predictions_y_round = np.rint(predictions_y)
predictions_y_round 

array([[2.],
       [2.],
       [1.],
       [2.],
       [2.],
       [2.],
       [1.],
       [0.],
       [1.],
       [2.],
       [2.],
       [2.],
       [1.],
       [1.],
       [2.],
       [0.],
       [2.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [2.],
       [2.],
       [2.],
       [1.],
       [1.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [2.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [1.],
       [2.],
       [3.],
       [0.],
       [2.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [2.],
       [2.],
       [0.],
       [0.],
       [1.],
       [1.],
       [2.],
       [0.],
       [2.],
       [1.],
       [2.],
       [2.],
       [1.],
       [0.],
       [2.],
       [0.],
       [2.],
       [1.],
       [2.],
       [2.],
       [2.],

In [18]:
# Viewing the number of predictions.
predictions_y_round.shape

(178, 1)

In [19]:
# Getting the sum of all the predictions obtained to used while obtaining FPA
s = 0
for  t in range(predictions_y_round.shape[0]):
    s+=predictions_y_round[t]
s

array([192.], dtype=float32)

In [20]:
# Obtaining the value of FPA metric for the model
Fpa = 0
for  t in range(predictions_y_round.shape[0]):
        x = 0
        for j in range( predictions_y_round.shape[0]-t+1, predictions_y_round.shape[0]):
               x = x + predictions_y_round[j]
        
        x = (x/s)/predictions_y_round.shape[0]
        Fpa = Fpa + x
Fpa

array([0.49177787], dtype=float32)

In [21]:
# Obtaining the value of CLC metric for the model
previous_obtained = predictions_y_round[predictions_y_round.shape[0] - 1]/s

CLC = 0
for i in range(predictions_y_round.shape[0]):
    if(i==0):
        CLC += 0 + previous_obtained
    else:
        additional = (predictions_y_round[predictions_y_round.shape[0] - 1 - i])/s
        CLC += 2*previous_obtained + additional
        previous_obtained += additional
        
CLC/=(2*predictions_y_round.shape[0])
CLC

array([0.50014627], dtype=float32)

In [22]:
# Getting direct metric results using the metrics given to model.
score = model.evaluate(test_x, test_y)
print(history.history.keys())
model.test_on_batch(test_x, test_y)
model.metrics_names
#print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[3], score[3]*100))


6/6 [==============================] - 0s 2ms/step - loss: 1.3362 - mse: 1.3362 - mae: 0.9334 - root_mean_squared_error: 1.1559 - mean_squared_logarithmic_error: 0.4605
dict_keys(['loss', 'mse', 'mae', 'root_mean_squared_error', 'mean_squared_logarithmic_error'])


['loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error']

In [23]:
# Saving the model
model_id = 9
path_to_save = '/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model4/ant1.3_ant1.4_model4_1'
model.save(path_to_save)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model4/ant1.3_ant1.4_model4_1/assets


In [24]:
# Writing the results to csv file.
heading = ['model_id', 'train_data_name', 'test_data_name'] + model.metrics_names + ['fpa', 'clc']
score = [model_id, train_data_name, test_data_name] + score + [float(Fpa) , float(CLC)]

In [25]:
heading

['model_id',
 'train_data_name',
 'test_data_name',
 'loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error',
 'fpa',
 'clc']

In [26]:
score

[9,
 'ant-1.3',
 'ant-1.4',
 1.3361612558364868,
 1.3361612558364868,
 0.9333840012550354,
 1.1559244394302368,
 0.46050211787223816,
 0.49177786707878113,
 0.5001462697982788]

In [27]:
# Writing the results to csv file.
with open(path_to_save + '_metric_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(heading)
    writer.writerow(score)


In [28]:
# Loading the model and to make sure that the model is saved properly.
model_loaded = load_model(path_to_save)